In [8]:
#Entropy model
import pandas as pd
import matplotlib as mtb
import numpy as np

In [9]:
origin = pd.read_excel('Origin_Livianos.xlsx')
destination = pd.read_excel('Destination_Livianos.xlsx')
matrix = pd.read_excel('Matrix_base3.xlsx')

In [10]:
## Check variableas and prepare base data
origin_zone = origin['Zone'].count()
destination_zone = destination['Zone'].count()

## Create Xm and Xn vector
Xm = [1]*origin_zone
XmPrev = [1]*origin_zone
Xn = [1]*destination_zone
XnPrev = [1]*destination_zone

dXm = [1]*origin_zone
dXn = [1]*destination_zone

## Prepare matrix

base_matrix = matrix.drop(columns = ['Zone']).values
base_matrix

array([[  0.        , 367.29602574,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,  45.18541715,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [177.61815191,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        , 198.06661642,
          0.        ,   0.        , 239.30918736, 262.43570547,
          0.        ,   0.        ,   0.        ],
       [ 25.24001289,  31.61509769,   0.        ,   0.        ,
          0.        ,  30.16258417,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 67.53408855,  84.59174787,   0.        ,   0.        ,
          0.        ,  80.70529278,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [ 29.16244733,  36.52

In [11]:
# Empty vectors

XnPrev[:] = Xn[:]
XmPrev[:] = Xm[:]

#While implementation
error = 2000
counter = 1
while (error >= 0.022):
    XnPrev[:] = Xn[:]
    XmPrev[:] = Xm[:]
    for n in range(0, destination_zone):
        Dn = destination['D_Trips'].loc[n]
        denM = 0
        for m in range(0, origin_zone):
            mSum = Xm[m]*base_matrix[m,n]
            denM = denM + mSum
        Xn[n] = Dn/denM
    Xn = np.nan_to_num(Xn, nan = 0)
    for m in range (0, origin_zone):
        Om = origin['O_Trips'].loc[m]
        denN = 0
        for n in range (0, destination_zone):
            nSum = Xn[n]* base_matrix[m,n]
            denN = denN + nSum
        Xm[m] = Om/denN   
    Xm = np.nan_to_num(Xm, nan = 0)
    ## Error estimation
    for n in range(0, destination_zone):
        dXn[n] = abs((Xn[n]-XnPrev[n])/XnPrev[n])
        dXn = np.nan_to_num(dXn, nan = 0)
        dXm[n] = abs((Xm[n]-XmPrev[n])/XmPrev[n])
        dXm = np.nan_to_num(dXm, nan = 0)
    errorN = np.mean(dXn)
    errorM = np.mean(dXm)
    error = np.maximum(errorN, errorM)

    print("Iteration: ", counter)
    print("Max average error", error)
    counter = counter + 1 

Iteration:  1
Max average error 1.1984620899082106e+307
Iteration:  2
Max average error 0.15320771039769912
Iteration:  3
Max average error 0.11581244395944225
Iteration:  4
Max average error 0.09264120983558762
Iteration:  5
Max average error 0.0758054605576937
Iteration:  6
Max average error 0.06319095178655615
Iteration:  7
Max average error 0.053580225792062605
Iteration:  8
Max average error 0.04613849777325055
Iteration:  9
Max average error 0.04076895143616665
Iteration:  10
Max average error 0.03656720810676069
Iteration:  11
Max average error 0.03318173129343694
Iteration:  12
Max average error 0.030569435673911162
Iteration:  13
Max average error 0.028798031041560295
Iteration:  14
Max average error 0.027737046270560686
Iteration:  15
Max average error 0.026974179514438302
Iteration:  16
Max average error 0.026322905174851374
Iteration:  17
Max average error 0.02576296708863003
Iteration:  18
Max average error 0.025278794771733535
Iteration:  19
Max average error 0.0248581865

<ipython-input-11-5c31d62015f7>:18: RuntimeWarning: invalid value encountered in true_divide
  Xn[n] = Dn/denM
<ipython-input-11-5c31d62015f7>:18: RuntimeWarning: divide by zero encountered in true_divide
  Xn[n] = Dn/denM
<ipython-input-11-5c31d62015f7>:26: RuntimeWarning: invalid value encountered in true_divide
  Xm[m] = Om/denN
<ipython-input-11-5c31d62015f7>:30: RuntimeWarning: invalid value encountered in double_scalars
  dXn[n] = abs((Xn[n]-XnPrev[n])/XnPrev[n])
<ipython-input-11-5c31d62015f7>:32: RuntimeWarning: invalid value encountered in double_scalars
  dXm[n] = abs((Xm[n]-XmPrev[n])/XmPrev[n])


In [12]:
### Create reception matrix

model_matrix = np.ones((origin_zone,destination_zone))
## fill matrix
for m in range (0, origin_zone):
    for n in range (0, destination_zone):
        model_matrix[m,n] = base_matrix[m,n]*Xn[n]*Xm[m]
model_matrix = pd.DataFrame(model_matrix)
model_matrix
model_matrix.to_excel("Matrix_final_test.xlsx")

In [ ]:
### Drafts
## Loop

XnPrev[:] = Xn[:]
XmPrev[:] = Xm[:]

for n in range(0, destination_zone):
    Dn = destination['D_Trips'].loc[n]
    denM = 0
    for m in range(0, origin_zone):
        mSum = Xm[m]*base_matrix[m,n]
        denM = denM + mSum
    Xn[n] = Dn/denM
Xn = np.nan_to_num(Xn, nan = 0)
for m in range (0, origin_zone):
    Om = origin['O_Trips'].loc[m]
    denN = 0
    for n in range (0, destination_zone):
        nSum = Xn[n]* base_matrix[m,n]
        print(Xn[n], base_matrix[m,n])
        denN = denN + nSum
    Xm[m] = Om/denN   
Xm = np.nan_to_num(Xm, nan = 0)


In [ ]:
## Error estimation
for n in range(0, destination_zone):
    dXn[n] = abs((Xn[n]-XnPrev[n])/XnPrev[n])
    print(dXn[n])



In [ ]:
np.mean(dXn)

In [ ]:
error

In [ ]:
XnPrev = Xn
XmPrev = Xm

In [ ]:
Xn

In [ ]:
XnPrev

In [ ]:
Xm

In [ ]:
base_matrix

In [ ]:
base_matrix[7,1]

In [ ]:
matrix